In [1]:
import math 
import pandas as pd
import random 
import numpy as np

In [2]:
# Setting simulation parameters values
N = [30, 50, 100, 500, 1000]  # population size
D = [0.01, 0.5, 1, 2] # Densities
d = 2 # distance in meters 
T = [1, 2, 5, 7, 10] # infectious time in days
iter = 100 # number of iterations
time = 1440 # exposure time in minutes
p0 = 0.0034 # enveloppe model hyperparameter 
lambda_ = 0.3712 # enveloppe model hyperparameter 
theta_ = math.pi/2 # a constant 

In [3]:
# Define the function to calculate the next values of r 

def f(theta_, r):
    return math.sqrt(r + d**2 - 2 * r * d * math.cos(theta_))

# Remark : I'll use this function later on to optimise the code, not used in this case !!!

In [4]:
%%time 

# Perform the simulations

data = []
current_row = None


for k in D: # density
    for n in N: # population size 
        for t in T: # infectious time
            
            for i in range(iter): # iterations
                results = 0
                
                for m in range(n): # individuals
                    mu = random.uniform(0, 1) # a random number between 0 and 1
                    r0 = math.sqrt((n * mu)/(k * math.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * math.exp(-lambda_ * r0)) 
                    proba_minute = p
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            r = math.sqrt(r + d**2 - 2 * r * d * math.cos(theta_))  # generate next radius distance per minute 
                            p = 1 - (p0 * math.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                            
                    proba_T = 1 - proba_minute ** t # apply the exponent on infectious time 
                    status = np.random.binomial(1, proba_T) # Bernoulli drawing for infectious status (0 or 1) using the final proba
                    results += status # sum on all Positives 

                # Check if the combination of parameters changes or still the same : to append number of positives as colomns instead of rows

                new_combination = current_row is None or (k, n, t) != (current_row["Density"], current_row["Population Size"], current_row["Infectious Time"])

                if new_combination:
                    # Create a new row
                    current_row = {"Density": k, "Population Size": n, "Infectious Time": t}
                    data.append(current_row)

                # Appending the new values of results as an additional columns
                current_row[f"Iteration {i}"] = str(results)
                    
print("Done")

Done
CPU times: total: 1h 19min 1s
Wall time: 1h 22min 29s


In [5]:
# Convert to dataframe for easy view
df = pd.DataFrame(data)

# Show the header (10 first lines)
df.head(10)

,Density,Population Size,Infectious Time,Iteration 0,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,...,Iteration 90,Iteration 91,Iteration 92,Iteration 93,Iteration 94,Iteration 95,Iteration 96,Iteration 97,Iteration 98,Iteration 99
0,0.01,30,1,29,25,24,19,25,25,27,...,27,27,27,24,23,28,27,27,24,26
1,0.01,30,2,29,30,30,30,30,28,29,...,30,30,30,30,30,30,30,30,29,29
2,0.01,30,5,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
3,0.01,30,7,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
4,0.01,30,10,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
5,0.01,50,1,44,43,41,35,39,45,48,...,39,41,41,40,44,42,46,45,42,41
6,0.01,50,2,47,49,49,50,50,49,50,...,49,49,49,49,50,49,49,47,50,49
7,0.01,50,5,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
8,0.01,50,7,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
9,0.01,50,10,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50


In [6]:
# Save the simulation results into a csv file

df.to_csv("simulation_results.csv", index=False)